In [1]:
import sys
import numpy as np
from random import randint
import random
import math
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
import operator
import sklearn.metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.naive_bayes import GaussianNB as GNB
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from deap import creator, base, tools, algorithms
from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()

In [2]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import gensim

In [3]:
model = gensim.models.KeyedVectors.load_word2vec_format('/Users/sukrverm/Documents/GoogleNews-vectors-negative300.bin', binary=True)

In [4]:
file1 = '/Users/sukrverm/Documents/NLP-Review/safecity/binary_classification/ogling_data/train.csv'
df = pd.read_csv(file1)
print(df.head())

corpus = list(df["Description"])
target = list(df["Category"])


                                         Description  Category
0  Was walking along crowded street, holding mums...         0
1  This incident took place in the evening.I was ...         1
2  I WAS WAITING FOR THE BUS. A MAN CAME ON A BIK...         0
3                 Incident happened inside the train         0
4  I witnessed an incident when a chain was bruta...         0


In [ ]:
infinity = float("inf") #sentinel maximum value
m = 70 #no of bats (50-70)
T = 100 # no of iterations(100)

bat_pos = defaultdict(list)
bat_vel = defaultdict(list)
bat_loudness = []
bat_rate = []
bat_fitness = []
bat_frequency = []

global_bat_pos = []
init_rate_vector = []
feature_vector = []

#hyper parameters
alpha = 0.9
gamma = 0.9
epsilon = random.uniform(-1,1)
fmin=0
fmax =1

In [5]:
#do once
#nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
vectorizer = TfidfVectorizer(ngram_range=(1, 3), stop_words=stop_words)
X = vectorizer.fit_transform(corpus)
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print(X.shape)
print(vectorizer.get_feature_names()[0:50])
print(len(vectorizer.get_feature_names()))

X_train = X
Y_train = target

file2 = '/Users/sukrverm/Documents/NLP-Review/safecity/binary_classification/ogling_data/dev.csv'
df_dev = pd.read_csv(file2)
print(df_dev.head())
corpus_dev = list(df_dev["Description"])
target_dev = list(df_dev["Category"])
X_dev = vectorizer.transform(corpus_dev)
print(X_dev.shape)
Y_dev = target_dev


(7201, 121356)
[u'00', u'00 confronted', u'00 confronted hit', u'00 go', u'00 go downstairs', u'00 way', u'00 way met', u'000', u'000 frs', u'000 frs could', u'000 frs parifoot', u'000 rupees', u'000 rupees man', u'00am', u'00am morning', u'00am morning tried', u'00pm', u'01', u'01 2011', u'01 2011 walking', u'03', u'03 2013', u'03 2013 scooty', u'03 29', u'03 29 2013', u'03 aa', u'03 aa 5562', u'0300', u'0300 pm', u'0300 pm time', u'05', u'05 2013', u'05 2013 evening', u'05 t6644', u'05 t6644 bus', u'06', u'06 2013', u'0740', u'0740 pm', u'0740 pm persons', u'0800am', u'0800am saw', u'0800am saw watchman', u'0830', u'0830 0900', u'0830 0900 sunday', u'08826862360', u'0900', u'0900 sunday', u'0900 sunday street']
121356
                                         Description  Category
0  Buses approaching to this place is highly unsafe.         0
1                        a man was commenting at me.         0
2                                    in a share auto         0
3  I was coming ou

In [6]:
def buildWordVector(tweet, size):
    vec = np.zeros(size).reshape((1, size))
    tokens = tokenize(tweet)
    print(tokens)
    count = 0.
    for word in tokens:
        try:
            vec += model.word_vec(word).reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [7]:
def tokenize(tweet):
    try:
        tweet = unicode(tweet.decode('utf-8').lower())
        tokens = tokenizer.tokenize(tweet)
        tokens = filter(lambda t: not t.startswith('@'), tokens)
        tokens = filter(lambda t: not t.startswith('#'), tokens)
        tokens = filter(lambda t: not t.startswith('http'), tokens)
        return tokens
    except:
        print('NC')
        return 'NC'

In [8]:
corpus_train_vecs_w2v = np.concatenate([buildWordVector(tweet, 300) for tweet in corpus])
corpus_dev_vecs_w2v = np.concatenate([buildWordVector(tweet, 300) for tweet in corpus_dev])

[u'was', u'walking', u'along', u'crowded', u'street', u',', u'holding', u'mums', u'hand', u',', u'when', u'an', u'elderly', u'man', u'groped', u'butt', u',', u'i', u'turned', u'to', u'look', u'at', u'h7m', u'and', u'he', u'looked', u'away', u',', u'and', u'did', u'it', u'again', u'after', u'a', u'while', u'.', u'i', u'was', u'12', u'yrs', u'old', u'then', u'.']
[u'this', u'incident', u'took', u'place', u'in', u'the', u'evening', u'.', u'i', u'was', u'in', u'the', u'metro', u'when', u'two', u'guys', u'started', u'staring', u'.']
[u'i', u'was', u'waiting', u'for', u'the', u'bus', u'.', u'a', u'man', u'came', u'on', u'a', u'bike', u'and', u'was', u'offering', u'liftvto', u'young', u'girls', u'.']
[u'incident', u'happened', u'inside', u'the', u'train']
[u'i', u'witnessed', u'an', u'incident', u'when', u'a', u'chain', u'was', u'brutally', u'snatched', u'from', u'an', u'elderly', u'lady', u'.', u'the', u'incident', u'took', u'place', u'during', u'evening', u'.']
[u'one', u'day', u'i', u'was'

[u'three', u'boys', u'normally', u'sit', u'at', u'a', u'spot', u'called', u'quotbiasharaquot', u'and', u'wait', u'for', u'me', u'to', u'pass', u'so', u'that', u'they', u'can', u'comment', u'on', u'my', u'dressing', u',', u'one', u'day', u'they', u'whistled', u'at', u'me', u'and', u'i', u'ignored.they', u'insulted', u'me']
[u'poor', u'lighting', u'when', u'walking', u'up', u'channing', u'to', u'turn', u'on', u'warring', u',', u'catcalling', u'as', u'well', u'.']
[u'no', u'street', u'light']
[u'this', u'incident', u'took', u'place', u'in', u'september', u'2012', u'during', u'morning', u'around', u'1011am', u'.', u'a', u'girl', u'who', u'was', u'driving', u'a', u'twowheeler.two', u'boys', u'hit', u'her', u'activa', u'scooty', u'softly', u'and', u'then', u'touched', u'her', u'back', u'and', u'swepting', u'clothes', u'.']
[u'i', u'was', u'travelling', u'in', u'a', u'bus', u'on', u'delhi', u'merut', u'highway', u'(', u'nh', u'58', u')', u'.', u'i', u'was', u'siting', u'somewhere', u'middle',

[u'a', u'man', u'around', u'0', u',', u'tried', u'to', u'get', u'hold', u'of', u'my', u'back', u'in', u'roadway', u'bus', u'while', u'i', u'was', u'travelling', u'back', u'to', u'my', u'hometown', u'.']
[u'taxi', u'driver', u'standing', u'outside', u'the', u'jalvayu', u'towers', u'apartment', u'was', u'flashing', u'at', u'10.00', u'am', u'.', u'confronted', u'and', u'hit', u'him', u'and', u'he', u'ran', u'away', u'.', u'doesnt', u'make', u'me', u'feel', u'better', u'.']
[u'stalking', u',', u'calling', u',', u'home', u'visits', u'.']
[u'when', u'i', u'was', u'returning', u'to', u'home', u'from', u'aunti', u'house', u'some', u'boys', u'in', u'micro', u'bus', u'was', u'commenting', u'and', u'touching', u'me', u'...']
[u'it', u'happened', u'on', u'the', u'station', u'while', u'i', u'was', u'waiting', u'for', u'a', u'train', u'a', u'man', u'starting', u'touching', u'himself', u'staring', u'at', u'all', u'us', u'women', u'.']
[u'i', u'am', u'a', u'frequent', u'traveller', u'of', u'mumbai', u

[u'this', u'incident', u'took', u'place', u'in', u'the', u'evening', u'of', u'7th', u'may', u'2013', u'.', u'a', u'friend', u'of', u'mine', u'when', u'came', u'out', u'of', u'the', u'club', u',', u'some', u'guys', u'started', u'commenting', u'on', u'her', u'dress', u'and', u'used', u'vulgar', u'language', u'and', u'even', u'followed', u'her', u'.']
[u'some', u'men', u'like', u'calling', u'us', u'and', u'tell', u'us', u'that', u'they', u'want', u'our', u'boobs', u'because', u'they', u'are', u'big']
[u'while', u'i', u'was', u'walking', u'on', u'the', u'street.two', u'boys', u'were', u'sitting', u'inside', u'the', u'cafe', u',', u'they', u'called', u'me', u'i', u"didn't", u'looked', u'at', u'them', u'and', u'kept', u'on', u'going', u',', u'one', u'of', u'the', u'boy', u'whistled', u'and', u'tried', u'to', u'follow', u'me.the', u'i', u'walked', u'as', u'fast', u'as', u'i', u'can', u'and', u'then', u'i', u'lost', u'his', u'sight', u'.']
[u'a', u'shopkeeper', u'started', u'touching', u'a', u

[u'this', u'incident', u'happened', u'in', u'a', u'public', u'vehicle', u'.', u'the', u'country', u'was', u'suffering', u'from', u'fuel', u'crisis', u'.', u'the', u'public', u'vehicles', u'were', u'packed', u'my', u'sister', u'was', u'travelling', u'in', u'a', u'micro', u'bus', u'where', u'there', u'was', u'no', u'space', u'to', u'put', u'in', u'the', u'legs', u'she', u'suddenly', u'saw', u'and', u'old', u'aged', u'man', u'(', u'abt', u'5560', u'yrs', u')', u'touching', u'the', u'hip', u'of', u'a', u'teenaged', u'girl', u'.', u'my', u'sister', u"didn't", u'say', u'anything', u'to', u'the', u'man', u'.', u'the', u'old', u'man', u'chronically', u'touched', u'the', u'hip', u'.', u'the', u'harrased', u'girl', u'noticed', u'the', u'man', u'and', u'returned', u'back', u'she', u'said', u'nothing', u'.']
[u'happened', u'in', u'afternoon']
[u'a', u'friend', u'of', u'mine', u'was', u'being', u'commented', u'upon', u'while', u'returning', u'from', u'her', u'tution', u'.']
[u'guys', u'masturbating

...

[u'buses', u'approaching', u'to', u'this', u'place', u'is', u'highly', u'unsafe', u'.']
[u'a', u'man', u'was', u'commenting', u'at', u'me', u'.']
[u'in', u'a', u'share', u'auto']
[u'i', u'was', u'coming', u'out', u'of', u'a', u'club', u'at', u'night', u'with', u'a', u'few', u'friends', u'when', u'a', u'few', u'guys', u'came', u'and', u'stopped', u'us', u'and', u'asked', u'quotkya', u'rate', u'hai', u'?', u'quot', u'when', u'we', u'shooed', u'them', u'away', u'they', u'kept', u'following', u'us', u'.']
[u'one', u'of', u'my', u'friends', u'was', u'molested', u'in', u'the', u'crowd', u'.', u'this', u'happened', u'on', u'no', u'9', u'bridge', u'of', u'the', u'thane', u'railway', u'station', u'.']
[u'i', u'was', u'travelling', u'in', u'metro', u'heading', u'to', u'my', u'college', u'.', u'a', u'guy', u'who', u'was', u'standing', u'next', u'to', u'me', u'tried', u'to', u'come', u'closer', u'to', u'me', u'and', u'even', u'gave', u'some', u'indecent', u'looks', u'.']
[u'a', u'guy', u'masturbat

[u'i', u'was', u'touched', u'inappropriately', u'touched', u'by', u'the', u'watchman', u'.']
[u'we', u'were', u'going', u'back', u'to', u'our', u'place', u'after', u'visiting', u'our', u'friend', u'.', u'so', u'as', u'we', u'were', u'going', u'to', u'the', u'ticket', u'counter', u'on', u'the', u'way', u'there', u'were', u'some', u'goons', u'kind', u'of', u'people', u'sitting', u'on', u'their', u'bikes', u'passed', u'comments', u'at', u'us', u'.']
[u'i', u'saw', u'a', u'young', u'man', u'harrasing', u'a', u'school', u'girl', u'touching', u'her', u'body']
[u'cheap', u'people', u'passing', u'comments', u'and', u'staring', u',', u'attempted', u'to', u'groping', u'too', u'.']
[u'my', u'friend', u'was', u'going', u'to', u'college', u'when', u'she', u'was', u'commented', u'at', u'by', u'a', u'few', u'boys', u'standing', u'there', u'.', u'this', u'happened', u'in', u'hii', u'block', u'of', u'madangir']
[u'commenting', u'amp', u'chain', u'snatching']
[u'i', u'was', u'hanging', u'out', u'with', 

[u'4', u'year', u'old', u'girl', u'playing', u'in', u'the', u'temple', u'with', u'her', u'friends', u'and', u'after', u'some', u'time', u'she', u'disappeared', u'and', u'after', u'2', u'hours', u'she', u'got', u'raped', u'.']
[u'i', u'stay', u'at', u'sindhi', u'colony', u',', u'sion', u'west', u',', u'opposite', u'to', u'old', u'sies', u'college', u'.', u'i', u'have', u'been', u'noticing', u'a', u'middle', u'aged', u'looking', u'beggar', u'in', u'a', u'lungi', u'standing', u'on', u'the', u'footpath', u'near', u'sies', u'college', u'and', u'singing', u'some', u'kawwali', u'or', u'something', u'.', u'2', u'weeks', u'back', u',', u'i', u'saw', u'him', u'standing', u'on', u'the', u'footpath', u'of', u'sies', u'college', u'and', u'singing', u'.', u'he', u'actually', u'tried', u'to', u'block', u'a', u'group', u'of', u'girls', u'with', u'his', u'hand', u'while', u'singing', u'.', u'i', u'was', u'like', u'why', u"don't", u'these', u'students', u'complain', u'to', u'the', u'college', u'security

[u'a', u'guy', u'trying', u'to', u'touch', u'me', u'at', u'inappropriate', u'places']
[u'there', u'were', u'two', u'boys', u'with', u'ice', u'cream', u'on', u'activa', u'i', u'was', u'not', u'wearing', u'helmet', u'and', u'was', u'on', u'a', u'scooty', u'.', u'they', u'tried', u'to', u'put', u'their', u'ice', u'cream', u'in', u'my', u'mouth', u'and', u'then', u'they', u'stalked', u'till', u'home', u'.']
[u'an', u'old', u'man', u'was', u'flashing', u'near', u'the', u'metro', u'station', u'entrance']
[u'during', u'night', u',', u'i', u'was', u'subjected', u'to', u'comments', u',', u'i', u'thought', u'it', u'was', u'best', u'to', u'ignore', u'the', u'comments', u'and', u'whistles', u'because', u'i', u'was', u'alone', u'and', u'there', u'was', u'no', u'one', u'around', u'.']
[u'around', u'6', u'months', u'back', u'a', u'boy', u'followed', u'me', u'in', u'the', u'afternoon', u'.']
[u'i', u'was', u'returning', u'from', u'my', u'dance', u'class', u'when', u'a', u'man', u'grabbed', u'my', u'ar

[u'someone', u'tried', u'to', u'strangle', u'me', u'.']
[u'like', u'every', u'other', u'morning', u'i', u'was', u'jogging', u'towards', u'the', u'central', u'park', u'when', u'two', u'boys', u'suddenly', u'started', u'jogging', u'behind', u'me', u'and', u'started', u'whistling', u'and', u'asking', u'me', u'to', u'look', u'back', u'at', u'them', u'as', u'they', u'made', u'kissing', u'sounds', u'.', u'there', u'were', u'no', u'people', u'on', u'the', u'streets', u'at', u'that', u'time', u'and', u'they', u'looked', u'pretty', u'drunk', u'to', u'me', u'.', u'they', u'hurled', u'abuses', u'and', u'asked', u'how', u'much', u'for', u'a', u'night', u'.', u'somehow', u'i', u'escaped', u'the', u'situation', u'but', u'when', u'is', u'this', u'all', u'every', u'going', u'to', u'stop', u'?', u'?']
[u'while', u'i', u'was', u'travelling', u'in', u'a', u'bus', u'from', u'ponda', u'to', u'keri', u',', u'the', u'conductor', u'of', u'that', u'bus', u'touched', u'my', u'breast', u'while', u'the', u'bus', 

In [9]:
print(corpus_dev_vecs_w2v.shape)
print(corpus_dev_vecs_w2v[0])

(990, 300)
[ 3.23062664e-01 -1.85195771e-01  4.66600823e-01 -1.88644573e-01
 -1.24973323e+00 -2.27244365e-01 -9.58952689e-01 -1.08181397e+00
  1.39345797e+00  1.75718093e-01  5.32842637e-01 -2.70843105e-01
  4.80081441e-01 -5.20783426e-01 -3.67876667e-01 -2.90576637e-02
 -1.26531293e+00 -3.28725784e-01 -2.39247423e-01  9.54430163e-02
  9.56674933e-01 -8.15628466e-01  1.07169272e-01  6.37225762e-02
  8.17776090e-01 -9.64660418e-01  1.40685546e-01  4.69624524e-01
  2.04635024e-02 -1.96768597e-01  1.79106545e-01 -1.11635262e-01
 -1.92194381e-01 -4.39475462e-02 -1.86683010e-02 -5.66644514e-01
 -1.42501345e-01 -2.53751528e-01 -3.43750806e-01 -5.75503629e-01
  2.11297521e-01 -6.07246393e-01  6.98544206e-01 -9.61333632e-02
 -8.52954116e-01 -1.60993430e-01 -1.36504352e-01  6.02337343e-01
 -6.37516066e-01  3.46394134e-01  1.09814855e-01  8.63003469e-01
 -2.52322561e-01 -1.42977756e-01 -2.04426587e-01  4.96794158e-01
  3.32260092e-01 -2.91613285e-01 -1.74130008e-01 -2.38709623e-01
 -5.42453304e-

In [10]:
#auxillary  functions
def extract_subset(z,cols):
    return z[:,cols]

In [11]:
def svm_calc_fitness(x):
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    #for j in range(0, X_train.shape[1]):
    for j in range(0, corpus_train_vecs_w2v.shape[1]):
        if x[j] == 1:
            cols.append(j)
    #z1_prime = extract_subset(X_train, cols)
    #z2_prime = extract_subset(X_dev, cols)
    z1_prime = extract_subset(corpus_train_vecs_w2v, cols)
    z2_prime = extract_subset(corpus_dev_vecs_w2v, cols)
    clf = svm.LinearSVC(C=1.0)
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_dev,y_pred)
    pre = sklearn.metrics.precision_score(Y_dev,y_pred)
    fscore = sklearn.metrics.f1_score(Y_dev,y_pred)
    recall = sklearn.metrics.recall_score(Y_dev,y_pred)
    return (acc),pre,fscore,recall



In [ ]:
def rf_calc_fitness(x):
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(z1, cols)
    z2_prime = extract_subset(z2, cols)
    clf = RandomForestClassifier(n_jobs=2,random_state=0)
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_test,y_pred)
    pre = sklearn.metrics.precision_score(Y_test,y_pred)
    fscore = sklearn.metrics.f1_score(Y_test,y_pred)
    recall = sklearn.metrics.recall_score(Y_test,y_pred)
    return (acc),pre,fscore,recall

In [ ]:
def knn_calc_fitness(x):
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(z1, cols)
    z2_prime = extract_subset(z2, cols)
    clf = KNN(3,weights='uniform')
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_test,y_pred)
    pre = sklearn.metrics.precision_score(Y_test,y_pred)
    fscore = sklearn.metrics.f1_score(Y_test,y_pred)
    recall = sklearn.metrics.recall_score(Y_test,y_pred)
    return (acc),pre,fscore,recall

In [ ]:
def nb_calc_fitness(x):
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(z1, cols)
    z2_prime = extract_subset(z2, cols)
    clf = GNB()
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_test,y_pred)
    pre = sklearn.metrics.precision_score(Y_test,y_pred)
    fscore = sklearn.metrics.f1_score(Y_test,y_pred)
    recall = sklearn.metrics.recall_score(Y_test,y_pred)
    return (acc),pre,fscore,recall

In [ ]:
def initialise(m,n):
    for i in range(0,m):
        for j in range(0,n):
            x = randint(0,1)
            bat_pos[i].append(x)
            bat_vel[i].append(0)

        init_rate_vector.append(random.uniform(0, 1))

        bat_frequency.append(init_rate_vector[i])
        bat_loudness.append(random.uniform(1,2))
        bat_rate.append(init_rate_vector[i])
        bat_fitness.append(-infinity)

In [ ]:
def core():
    initialise(m,n)
    global_fit = -infinity
    for t in range (0,T):
        print("T:" , t)
        for i in range(0,m):
            accuracy,pre,fscore,recall = svm_calc_fitness(bat_pos[i])
            '''
            with open("dumpPerformanceBat_svm_kpi.txt", "a") as text_file:
                text_file.write("{} {} {} {} {}".format(accuracy,pre,fscore,recall,global_fit) + "\n")
            with open("dumpPerformanceBat_svm.txt", "a") as text_file:
                text_file.write("{} {} {} {}".format(t,accuracy,bat_pos[i].count(1),global_fit)+"\n")
            #print("t={} accuracy={} #features={} global_fit={}".format(t,accuracy,bat_pos[i].count(1),global_fit))
            '''
            acc = accuracy

            rand = randint(0,1)
            if rand < bat_loudness[i] and acc > bat_fitness[i]:
                bat_fitness[i]=acc
                bat_loudness[i]= alpha * bat_loudness[i]
                bat_rate[i]= init_rate_vector[i] * (1 - math.exp(-(gamma * t)))

        maxindex,maxfit = max(enumerate(bat_fitness), key=operator.itemgetter(1))

        if maxfit>global_fit :
            global_fit = maxfit
            global_bat_pos = bat_pos[maxindex]

        for i in range(0,m): # for each bat
            beta = random.uniform(0,1)
            rand = random.uniform(0,1)
            AvgA = sum(bat_loudness)/len(bat_loudness)

            if rand > bat_rate[i] :
                for j in range(0,n):
                    bat_pos[i][j] = bat_pos[i][j] + ( epsilon * AvgA )
                    sigma = randint(0,1)
                    if sigma < (1/(1+math.exp(-bat_pos[i][j]))) :
                        bat_pos[i][j]=1
                    else :
                        bat_pos[i][j]= 0

            rand = random.uniform(0,1)
            if rand < bat_loudness[i] and bat_fitness[i] < global_fit :
                for j in range (0,n):
                    bat_frequency[i] = fmin + (fmax-fmin)* beta
                    bat_vel[i][j] = bat_vel[i][j] + (global_bat_pos[j]-bat_pos[i][j]) * bat_frequency[i]
                    bat_pos[i][j] = bat_pos[i][j] + bat_vel[i][j]
                    sigma = random.uniform(0,1)
                    if sigma < (1 / (1 + math.exp(-bat_pos[i][j]))) :
                        bat_pos[i][j]=1
                    else :
                        bat_pos[i][j]=0
        with open("dumpPerformanceBat_rf.txt", "a") as text_file:
            text_file.write("{} {} {} ".format(t,global_bat_pos.count(1),global_fit)+"\n")
        


    feature_vector = global_bat_pos
    return feature_vector

In [ ]:
def BAT_algo():
    feature_vector = core()
    return feature_vector

In [ ]:
f = BAT_algo()
 

In [12]:
class GeneticAlgorithm():
    
    def __init__(self, X_train, X_test, Y_train, Y_test, number_bits_to_mutate = 25.0):
        
        
        self.toolbox = base.Toolbox()
        self.ind_mut_pb = float(number_bits_to_mutate / X_train.shape[1])
        self.NGEN = 100
        self.MUTPB = 0.2
        self.CXPB = 0.5
        self.POPULATION_SIZE = 70
        self.NUMBER_TO_SELECT_FROM_POP = 35
        self.init_pop = list()
        self.X_tr = X_train
        self.X_te = X_test
        self.Y_train = Y_train
        self.Y_test = Y_test
        
    
    def initialise_toolbox(self):
        print('now initializing toolbox GA1')
        # I want to maximize session overlap as well as length
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        # representing individual as a list
        creator.create("Individual", list, fitness=creator.FitnessMax)
        # individual will be a list of 0/1
        self.toolbox.register("attr_bool", random.randint, 0, 1)
        IND_DIMENSIONALITY = self.X_tr.shape[1]
        # create template for individual
        self.toolbox.register("individual", tools.initRepeat, creator.Individual, self.toolbox.attr_bool, n=IND_DIMENSIONALITY)
        #self.toolbox.register("individual", tools.initIterate, creator.Individual, self.get_ind)
        # create template for population
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        print('toolbox initialised')
    
    
    def initialise_fitness_function(self):
        print('building and registering the fitness function')
        
        def fitness(individual):
            
            
            accuracy,pre,fscore,recall = svm_calc_fitness(individual)
            return fscore,
            
        
        self.toolbox.register("evaluate", lambda individual: fitness(individual))
        self.toolbox.register("mate", tools.cxOnePoint)
        self.toolbox.register("mutate", tools.mutFlipBit, indpb=self.MUTPB)
        self.toolbox.register("select", tools.selTournament, tournsize=3)
        
        print('fitness function ready')
    
    def run_algorithm(self):
        
        self.initialise_toolbox()
        self.initialise_fitness_function()
        
        print('mutation probability was calculated at %f' % self.ind_mut_pb)
        print('running the algorithm')
        toolbox = self.toolbox
        # create a population of size n
        pop = toolbox.population(n=self.POPULATION_SIZE)
        
        
        for gen in range(self.NGEN):
            # Select the next generation individuals
            if gen%5 == 0:
                print(gen)
            offspring = toolbox.select(pop, len(pop))
            
            for ind in offspring:
                if ind.fitness.valid:
                    pass
            # Clone the selected individuals
            offspring = map(toolbox.clone, offspring)

            # Apply crossover on the offspring
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < self.CXPB:
                    #toolbox.mate(child1, child2)
                    tools.cxUniform(child1, child2, 0.5)
                    del child1.fitness.values
                    del child2.fitness.values

            # Apply mutation on the offspring
            for mutant in offspring:
                if random.random() < self.MUTPB:
                    # print('now doing mutation')
                    toolbox.mutate(mutant)
                    del mutant.fitness.values

            # Evaluate the individuals with an invalid fitness
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
            
            added = 0
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit
                

            # The population is entirely replaced by the offspring
            pop[:] = offspring
            top = tools.selBest(pop, k=self.NUMBER_TO_SELECT_FROM_POP)
            
            # Dump best individual to file
            fittest = tools.selBest(pop, k=1)[0]
            count = 0
            for i in range(0, len(fittest)):
                if fittest[i] == 1:
                    count = count + 1 
            with open("GA_performance.txt", "a") as text_file:
                accuracy,prec,fscore,recall = svm_calc_fitness(fittest)
                text_file.write("{} {}".format(count,fittest.fitness.values[0])+"\n")
                print(count,accuracy,prec,fscore,recall,fittest.fitness.values[0])
        
        
        
        
        max_fitness = float("-inf")
        
        for individual in tools.selBest(pop, k=1):
            if individual.fitness.valid:
                if individual.fitness.values[0] > max_fitness:
                    max_fitness = individual.fitness.values[0]
                    fittest_individual = individual
                    
        count = 0
        for i in range(0, len(fittest_individual)):
            if fittest_individual[i] == 1:
                count = count + 1        
        
        print(count, max_fitness)
        return fittest_individual

In [13]:
ga = GeneticAlgorithm(corpus_train_vecs_w2v, corpus_dev_vecs_w2v, Y_train, Y_dev)

In [14]:
fittest_individual = ga.run_algorithm()

now initializing toolbox GA1
toolbox initialised
building and registering the fitness function
fitness function ready
mutation probability was calculated at 0.083333
running the algorithm
0
(148, 0.8232323232323232, 0.7375, 0.40273037542662116, 0.27699530516431925, 0.40273037542662116)
(153, 0.8282828282828283, 0.7945205479452054, 0.40559440559440557, 0.27230046948356806, 0.4027777777777778)
(152, 0.8272727272727273, 0.7625, 0.41638225255972694, 0.2863849765258216, 0.41780821917808225)
(149, 0.8313131313131313, 0.8194444444444444, 0.41403508771929826, 0.27699530516431925, 0.41811846689895477)
(156, 0.8303030303030303, 0.8, 0.4166666666666667, 0.28169014084507044, 0.4166666666666667)
5
(147, 0.8303030303030303, 0.7777777777777778, 0.42857142857142855, 0.29577464788732394, 0.42857142857142855)
(163, 0.8323232323232324, 0.7901234567901234, 0.435374149659864, 0.3004694835680751, 0.435374149659864)
(162, 0.8313131313131313, 0.7738095238095238, 0.4377104377104377, 0.3051643192488263, 0.43771

(166, 0.8393939393939394, 0.8068181818181818, 0.4717607973421927, 0.3333333333333333, 0.4717607973421927)
(166, 0.8393939393939394, 0.8068181818181818, 0.4717607973421927, 0.3333333333333333, 0.4717607973421927)
(164, 0.8393939393939394, 0.8068181818181818, 0.4717607973421927, 0.3333333333333333, 0.4717607973421927)
(165, 0.8393939393939394, 0.8068181818181818, 0.4717607973421927, 0.3333333333333333, 0.4717607973421927)
80
(166, 0.8393939393939394, 0.8068181818181818, 0.4717607973421927, 0.3333333333333333, 0.4717607973421927)
(166, 0.8393939393939394, 0.8068181818181818, 0.4717607973421927, 0.3333333333333333, 0.4717607973421927)
(166, 0.8393939393939394, 0.8068181818181818, 0.4717607973421927, 0.3333333333333333, 0.4717607973421927)
(164, 0.8393939393939394, 0.8068181818181818, 0.4717607973421927, 0.3333333333333333, 0.4717607973421927)
(164, 0.8393939393939394, 0.8068181818181818, 0.4717607973421927, 0.3333333333333333, 0.4717607973421927)
85
(166, 0.8393939393939394, 0.806818181818

In [15]:
file3 = '/Users/sukrverm/Documents/NLP-Review/safecity/binary_classification/ogling_data/test.csv'
df_test = pd.read_csv(file3)
print(df_test.head())
corpus_test = list(df_test["Description"])
target_test = list(df_test["Category"])
X_test = vectorizer.transform(corpus_test)
print(X_test.shape)
print(vectorizer.get_feature_names()[0:50])
print(len(vectorizer.get_feature_names()))
Y_test = target_test

                                         Description  Category
0  During morning, a woman was walking by and thi...         1
1  A man tried to brush his penis off of a woman'...         0
2  This happened to a fellow passenger of mine tr...         1
3                                             ogling         1
4  When I was returning my home after finishing m...         0
(1701, 121356)
[u'00', u'00 confronted', u'00 confronted hit', u'00 go', u'00 go downstairs', u'00 way', u'00 way met', u'000', u'000 frs', u'000 frs could', u'000 frs parifoot', u'000 rupees', u'000 rupees man', u'00am', u'00am morning', u'00am morning tried', u'00pm', u'01', u'01 2011', u'01 2011 walking', u'03', u'03 2013', u'03 2013 scooty', u'03 29', u'03 29 2013', u'03 aa', u'03 aa 5562', u'0300', u'0300 pm', u'0300 pm time', u'05', u'05 2013', u'05 2013 evening', u'05 t6644', u'05 t6644 bus', u'06', u'06 2013', u'0740', u'0740 pm', u'0740 pm persons', u'0800am', u'0800am saw', u'0800am saw watchman', u'0830'

In [16]:
corpus_test_vecs_w2v = np.concatenate([buildWordVector(tweet, 300) for tweet in corpus_test])

[u'during', u'morning', u',', u'a', u'woman', u'was', u'walking', u'by', u'and', u'thin', u'guy', u'came', u'around', u'and', u'called', u'her', u'names', u',', u'and', u'commented', u'on', u'her', u'figure', u'.', u'the', u'woman', u'handled', u'him', u'and', u'abused', u'.']
[u'a', u'man', u'tried', u'to', u'brush', u'his', u'penis', u'off', u'of', u'a', u"woman's", u'shoulder', u'in', u'the', u'bus', u'.']
[u'this', u'happened', u'to', u'a', u'fellow', u'passenger', u'of', u'mine', u'travelling', u'in', u'the', u'metro', u'.', u'he', u'was', u'kicked', u'out', u'of', u'the', u'metro', u'station', u'because', u'he', u'was', u'quotdifferentlyquot', u'dressed', u'.']
[u'ogling']
[u'when', u'i', u'was', u'returning', u'my', u'home', u'after', u'finishing', u'my', u'class', u'.', u'i', u'was', u'in', u'queue', u'to', u'get', u'on', u'the', u'micro', u'bus', u'and', u'there', u'was', u'a', u'girl', u'opposite', u'to', u'me', u'just', u'then', u'a', u'young', u'man', u'tried', u'to', u'tou

[u'had', u'to', u'use', u'this', u'subway', u'yesterday', u'at', u'5pm', u',', u'was', u'walking', u'with', u'my', u'son', u',', u'it', u'was', u'very', u'embarrassing', u',', u'two', u'older', u'men', u',', u'kind', u'of', u'beggars', u',', u'made', u'gestures', u'and', u'exposures', u'.']
[u'few', u'men', u',', u'sitting', u'on', u'their', u'bikes', u'kept', u'stalking', u'our', u'group', u'.', u'as', u'soon', u'as', u'a', u'police', u'van', u'was', u'passing', u'by', u'we', u'decided', u'to', u'report', u'but', u'before', u'we', u'could', u',', u'they', u'fled', u'from', u'the', u'spot', u'.']
[u'there', u'are', u'so', u'many', u'incidents', u'i', u'have', u'experienced', u'but', u'the', u'one', u'i', u'want', u'to', u'write', u'here', u'happened', u'a', u'year', u'ago', u'when', u'a', u'girl', u'of', u'45', u'years', u'who', u'was', u'been', u'raped', u'by', u'his', u'own', u'neighbor', u'and', u'her', u'parents', u'took', u'her', u'to', u'the', u'same', u'hospital', u'where', u'i'

[u'once', u'i', u'off', u'to', u'the', u'market', u'to', u'buy', u'stuff', u'when', u'i', u'sensed', u'someone', u'following', u'me', u'i', u'found', u'a', u'guy', u'secretly', u'following', u'me', u'and', u'whenever', u'i', u'look', u'at', u'him', u'he', u'would', u'pretend', u'using', u'his', u'phone', u'then', u'i', u'had', u'to', u'take', u'an', u'alternate', u'route', u'.']
[u'catcalls', u'/', u'whistles', u'in', u'sarojini', u'nagar', u'market', u'(', u'evening', u'time', u')']
[u'nearby', u'this', u'place', u'of', u'clock', u'tower', u',', u'there', u'is', u'kamla', u'nagar', u'market', u'where', u'oftenly', u'girls', u'go', u'for', u'shopping', u',', u'on', u'that', u'very', u'day', u'two', u'boys', u'from', u'the', u'roadside', u'came', u'and', u'started', u'passing', u'bad', u'comments', u'on', u'the', u'girls', u'nearby', u'by', u'momo', u'corner', u'and', u'pushed', u'a', u'girl', u'while', u'passing', u'that', u'place', u'.']
[u'in', u'a', u'huge', u'crowd', u'at', u'rajiv

[u'i', u'was', u'followed', u'home']
[u'in', u'night', u'2', u'boys', u'just', u'making', u'nonsense', u'things', u'on', u'the', u'road']
[u'tried', u'to', u'touch', u'me', u'and', u'followed', u'me', u'at', u'night', u'time', u'.']
[u'in', u'the', u'bus']
[u'gold', u'chain', u'was', u'snatched', u'.']
[u'during', u'afternoon', u',', u'while', u'roaming', u'i', u'was', u'commented', u'.']
[u'teasing', u'while', u'walking', u'on', u'the', u'street']
[u'during', u'afternoon', u',', u'a', u'guy', u'sitting', u'in', u'front', u'of', u'me', u'in', u'metro', u'kept', u'clicking', u'mi', u'pictures', u'.']
[u'i', u'was', u'walking', u'with', u'a', u'friend', u'in', u'south', u'mumbai', u'and', u'a', u'man', u'brushed', u'past', u'us', u'.', u'we', u'turned', u'back', u'and', u'created', u'quite', u'a', u'scene', u'.', u'man', u'crossed', u'the', u'road', u'and', u'started', u'ignoring', u'!', u'!']
[u'indecently', u'touched', u'in', u'public', u'transport']
[u'it', u'was', u'really', u'bad', 

[u'i', u'was', u'called', u'names', u'when', u'i', u'was', u'alone', u'passing', u'by', u'leh', u'market']
[u'i', u'was', u'walking', u'on', u'the', u'road', u'to', u'reach', u'bus', u'stand', u',', u'there', u'stood', u'a', u'bunch', u'of', u'guys', u'whistling', u'passing', u'vulgar', u'comments', u'at', u'me', u'.', u'i', u'ignored', u'and', u'just', u'went', u'passed', u'them', u',', u'they', u'followed', u'me', u'n', u'continued', u'doing', u'the', u'same.its', u'becoming', u'difficult', u'for', u'women', u'to', u'survive', u'.', u'please', u'wake', u'up', u'and', u'do', u'something', u'about', u'it', u'.']
[u'i', u'was', u'spanked', u'by', u'a', u'woman', u'in', u'koinange', u'street', u'while', u'passing']
[u'when', u'i', u'was', u'making', u'a', u'call', u'my', u'father', u'came', u'and', u'seized', u'my', u'phone', u'.', u'i', u'was', u'very', u'angry', u'and', u'left', u'the', u'house', u'because', u'it', u'was', u'not', u'good', u'for', u'me', u'to', u'see', u'my', u'father'

...

[u'ogling', u'and', u'commenting']
[u'my', u'sister', u'studies', u'away', u'from', u'home', u'.', u'she', u'stays', u'in', u'a', u'separate', u'house', u'near', u'her', u'college', u'.', u'she', u'had', u'her', u'college', u'mates', u'at', u'her', u'place', u'for', u'late', u'night', u'studies', u'and', u'one', u'by', u'one', u'all', u'left', u'except', u'her', u'boyfriend', u'.', u'and', u'it', u'so', u'happened', u'eventually', u'he', u'raped', u'her', u'that', u'night', u'.']
[u'inside', u'a', u'microbus', u',', u'i', u'was', u'touched', u'by', u'a', u'man', u'physically', u'and', u'i', u'became', u'so', u'nervous', u'.']
[u'night', u'8', u'pm']
[u'boys', u'were', u'passing', u'commnets']
[u'2', u'men', u'approached', u'me', u'in', u'the', u'market', u'to', u'go', u'to', u'the', u'nearest', u'hotel', u'.', u'just', u'because', u'i', u'am', u'a', u'foreigner', u',', u'they', u'thought', u'i', u'am', u'available']
[u'a', u'friend', u'of', u'mine', u'was', u'going', u'for', u'a', u'wa

[u'commenting', u',', u'touching', u',', u'groping']
[u'due', u'to', u'poor', u'street', u'lightning', u'cases', u'like', u'theft', u'and', u'robbery', u'took', u'place']
[u'this', u'incident', u'took', u'place', u'during', u'afternoon.three', u'men', u'tried', u'to', u'pull', u'me', u'into', u'the', u'car', u'just', u'outside', u'my', u'college', u'.', u'i', u'reported', u'to', u'the', u'college', u'department', u'and', u'even', u'to', u'nearby', u'pcr', u'van', u'but', u'all', u'in', u'vain.the', u'incident', u'has', u'happened', u'to', u'my', u'other', u'friends', u'also', u'.']
[u'i', u'was', u'in', u'a', u'rickshaw', u'and', u'two', u'boys', u'started', u'commenting', u'on', u'my', u'clothes.it', u'happened', u'near', u'gol', u'chakkar', u'in', u'the', u'evening', u'.']
[u'a', u'man', u'tried', u'to', u'touch']
[u'some', u'group', u'of', u'idle', u'boys', u'sit', u'at', u'usalama', u'bridge', u'that', u'connects', u'lindi', u'and', u'laini', u'saba', u'harassing', u',', u'chasing'

[u'chain', u'snatching']
[u'some', u'guys', u'were', u'commenting', u'the', u'girls', u'.']
[u'two', u'or', u'more', u'boys', u'were', u'sitting', u'at', u'the', u'chowk', u'and', u'they', u'were', u'commenting', u'on', u'the', u'dress', u'of', u'the', u'girl', u'.']
[u'once', u'i', u'was', u'coming', u'back', u'from', u'my', u'college', u'and', u'saw', u'a', u'man', u'masturbating', u'in', u'his', u'car', u'with', u'doors', u'widely', u'open.he', u'was', u'watching', u'the', u'passersby', u'but', u'it', u'had', u'no', u'effect', u'on', u'his', u'activity', u'.', u'this', u'repeated', u'once', u'again', u'and', u'when', u'enquired', u'feom', u'others', u',', u'i', u'got', u'to', u'know', u'that', u'it', u'happens', u'there', u'aa', u'a', u'daily', u'chore', u'.']
[u'someone', u'stole', u'my', u'wallet', u'during', u'evening', u'time', u'.']
[u'someone', u'groped', u'my', u'butt', u'while', u'i', u'was', u'trying', u'to', u'get', u'out', u'of', u'the', u'metro', u'.']
[u'i', u'was', u'w

[u'whistled', u'at', u'by', u'guy', u'at', u'a', u'tea', u'stall', u',', u'other', u'men', u'around', u'were', u'laughing', u'and', u'enjoying', u'it']
[u'teasing', u'with', u'different', u'names']
[u'walking', u'on', u'the', u'street', u'will', u'elicit', u'unwanted', u'attention', u',', u'in', u'the', u'form', u'of', u'sexual', u'objectification', u'through', u'commentary', u'and', u'ogling', u'.']
[u'a', u'man', u'i', u'know', u'incists', u'on', u'taking', u'me', u'to', u'bed', u'but', u'i', u'fear', u'telling', u'him', u'off']
[u'teacher', u'beating', u'student']
[u'a', u'guy', u'was', u'trying', u'to', u'be', u'physical', u'with', u'one', u'of', u'my', u'friend', u'in', u'nepal', u'yatayat', u'.']
[u'while', u'deboarding', u'the', u'metro', u'at', u'rajiv', u'chowk', u',', u'a', u'young', u'guy', u'rubbed', u'me', u'off', u'twice', u'.', u'initially', u'i', u'ignored', u'it', u'thinking', u'it', u'is', u'due', u'to', u'overcrowded', u'metro', u'.', u'but', u'when', u'it', u'kept',

[u'this', u'incident', u'took', u'place', u'in', u'sepetember', u'2013', u'in', u'the', u'evening.while', u'i', u'was', u'travelling', u'to', u'subhash', u'nagar', u',', u'on', u'the', u'way', u'i', u'had', u'to', u'cross', u'raghubir', u'nagar', u'too', u',', u'the', u'local', u'people', u'who', u'were', u'outside', u'the', u'slums', u'started', u'commenting', u'and', u'also', u'ogled', u'at', u'me', u'.']
[u'a', u'group', u'of', u'girl', u'were', u'passing', u'through', u'the', u'hostel', u'gate', u',', u'the', u'seniors', u'girls', u'were', u'commenting', u'on', u'us', u'and', u'trying', u'to', u'bully', u'us', u'.']
[u'the', u'place', u'is', u'safe', u'eough', u'but', u'doesnt', u'have', u'a', u'proper', u'transportation', u'service']
[u'the', u'people', u'liviingin', u'the', u'sluma', u'rea', u'near', u'the', u'station', u'make', u'it', u'really', u'unsafe', u'for', u'people', u'to', u'walk', u'at', u'night']
[u'followed', u'by', u'a', u'guy', u'in', u'my', u'colony']
[u'north', u

In [17]:
def svm_calc_final_metrics(x):
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    print(X_test.shape[0])
    #for j in range(0, X_train.shape[1]):
    for j in range(0, corpus_train_vecs_w2v.shape[1]):
        if x[j] == 1:
            cols.append(j)
    #z1_prime = extract_subset(X_train, cols)
    #z2_prime = extract_subset(X_test, cols)
    z1_prime = extract_subset(corpus_train_vecs_w2v, cols)
    z2_prime = extract_subset(corpus_test_vecs_w2v, cols)
    clf = svm.LinearSVC(C=1.0)
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_test,y_pred)
    pre = sklearn.metrics.precision_score(Y_test,y_pred)
    fscore = sklearn.metrics.f1_score(Y_test,y_pred)
    recall = sklearn.metrics.recall_score(Y_test,y_pred)
    return (acc),pre,fscore,recall

In [18]:
accuracy,prec,fscore,recall = svm_calc_final_metrics(fittest_individual)
print(accuracy,prec,fscore,recall,fittest_individual.fitness.values[0])

1701
(0.807172251616696, 0.7155963302752294, 0.3223140495867769, 0.208, 0.4717607973421927)
